[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ferit-osirv/lab6-home/blob/main/lab6_home.ipynb)


## Lab 6: Feature Detection and Image Retrieval

In this lab, you will develop an image retrieval algorithm. Image retrieval involves finding images within a large dataset that match a given query image. For example, if the query image features the Eiffel Tower, the image retrieval algorithm should identify and retrieve all images containing the Eiffel Tower.

**Note:** Depending on your group ID, you will later be assigned a specific landmark to track.

## Resources

Most of the resources required for this assignment can be found in this guide:

[OpenCV Documentation: Feature Detection and Description](https://docs.opencv.org/4.x/db/d27/tutorial_py_table_of_contents_feature2d.html)

Please read through all sections, paying particular attention to the section relevant to your assigned method and the last two sections. The last two sections will be essential for implementing this assignment.

## Image Retrieval Algorithm

The algorithm will operate as follows:

**Step 1:** Perform feature matching between each query image and every search image using a feature detection algorithm such as FAST, SIFT, etc.

**Step 2:** If there are more than a certain number of matches (e.g., 10) between a query and a search image, we can conclude that the search image contains the landmark. This result is stored as either true or false.

**Step 3:** Calculate the true positive and false positive rates to evaluate the algorithm's performance.

**Algorithm in Pseudocode**

Input: A list of query images featuring a specific landmark, a list of images for searching, some of which contain the landmark and some that do not, and a threshold value (e.g., 10) for the number of matches required to confirm the presence of the landmark in an image.

```python
image_contains_landmark = []
for each image with index i:
  image_contains_landmark[i] = false

for each query image:
  for each image with index i:
    matches = match_features(query, image)
    if matches count > threshold:
      image_contains_landmark[i] = true
```

**Feature Matching**

The feature matching process should proceed in several steps (as explained in the "Feature Matching" section referenced above):

**Step 1:** Conduct feature detection on the query image using **ORB**.

**Step 2:** Perform feature detection on the search image using **ORB**.

**Step 3:** Use **FLANN (Fast Approximate Nearest Neighbor Search Library)** with Lowe's ratio test to match the features of the query and search image.

**Step 4:** Implement **Lowe's ratio test** to retain only features below a certain distance threshold.

**Step 5:** Return the matches that are above the distance threshold.

## Dataset Downloading Code

The code block below downloads the following dataset:

https://github.com/filipradenovic/revisitop (rparis6k)
published in: Radenović, F., Iscen, A., Tolias, G., Avrithis, Y., & Chum, O. (2018). Revisiting Oxford and Paris: Large-Scale Image Retrieval Benchmarking. 2018 IEEE/CVF Conference on Computer Vision and Pattern Recognition, 5706-5715.

The dataset contains thousands of images of various landmarks in Paris, France. It also contains query images for each landmark, and labels of which landmark is on each image.

The code block also defines helper functions which will be explained later in the notebook.

**⚠️ Click the ▶️ button to run the cell below.**

In [ ]:
import os
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import shutil
import os.path as p

# clone git repository
if not os.path.exists('images'):
    !git clone https://github.com/ferit-osirv/lab6-home.git
    shutil.move('lab6-home/images', 'images')
    shutil.rmtree('lab6-home')

def get_query_img(landmark):
  fname = p.join('images', 'query', landmark + '.jpg')
  img = cv.imread(fname)
  img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
  return img

def get_imgs(landmark):
  imgs = []
  folder = p.join('images', landmark)
  for fname in os.listdir(folder):
    if fname.split('.')[-1] != 'jpg':
      continue
    img = cv.imread(p.join(folder, fname))
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    imgs.append(img)
  return imgs

def get_img_fnames(landmark):
  fnames = []
  folder = p.join('images', landmark)
  for fname in os.listdir(folder):
    if fname.split('.')[-1] != 'jpg':
      continue
    fnames.append(p.join(folder, fname))
  return fnames

landmarks = ['eiffel', 'louvre', 'defense', 'moulinrouge', 'invalides']

all_img_fnames = []
for landmark in landmarks:
  all_img_fnames += get_img_fnames(landmark)

num_imgs = len(all_img_fnames)

def get_img_fname(i):
  return all_img_fnames[i]

def get_img(i):
  img = cv.imread(get_img_fname(i))
  img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
  return img

## Groups

In the notice of this lab, you received a group number.

**⚠️ Enter the assignment group id below and run the block get your landmark and method.**

In [ ]:
# @title  { run: "auto", vertical-output: true }
group_id = 0 # @param {type:"integer"}
assert len(str(group_id)) == 1, "Incorrect group"

landmarks = ['pantheon', 'louvre', 'notredame', 'triomphe', 'eiffel']
methods = ['SIFT', 'ORB', 'FAST']

assert (group_id - 1) >= 0 and (group_id - 1) < len(landmarks), "Incorrect group"

landmark = landmarks[group_id - 1]
print("Your landmark:", landmark)

## Documentation

**Available predefined functions in this notebook**:


 - `get_query_imgs(landmark)` - load a list of query images
   - `landmark` can be `'pantheon', 'louvre', 'notredame', 'triomphe', 'eiffel'`.
 - `get_img(i)` - load the `i`th image from the dataset
 - `get_img_fname(i)` - returns the file name of the image
 - `num_imgs` - the total number of images in the dataset
 - `get_imgs(landmark)` - get all images of a certain landmark, useful for evaluation
 - `get_img_fnames(landmark)` - get all image filenames of a certain landmark, useful for evaluation

In [ ]:
# Some example code

query_img = get_query_img('eiffel')
plt.imshow(query_img)
plt.title('Example Query');
plt.show()

example_img = get_img(40)
plt.imshow(example_img)
plt.title('Example Image');
plt.show()

example_eiffel_img = get_imgs('eiffel')[5]
plt.imshow(example_eiffel_img)
plt.title('Example Image of the Eiffel Tower');
plt.show()

print('Image 40', get_img_fname(40))
print('All Eiffel images', get_img_fnames('eiffel'))

## Assignment

In [ ]:
print(landmark)

## Initial Algorithm Development

For algorithm development, it is faster to only go through known images of that landmark.

Later in the notebook, you will perform the search across all images.

Before you start development here, it might be a good idea to first try implementing feature matching for just one query and one search image. You can then convert that code to a for loop to go through multiple images as in the block below.

In [ ]:
# TODO: Implement ORB by detecting keypoints and computing descriptors
#      for the query and the search image. Match the descriptors
#      using a FLANN-based matcher. Use cv.drawMatchesKnn to
#      visualize the matches. You can use the provided materials as a reference.

query_img = get_query_img(landmark)
search_img = get_imgs(landmark)[0]

# Note: Detect the features on grayscale images.



Next, we can compute the features across all images and store them in a list.

In [ ]:
# For this initial phase, we will only go through known landmarks.
# Later in this notebook, we will go through every image in the dataset.
known_landmark_fnames = get_img_fnames(landmark)
known_landmark_imgs = get_imgs(landmark)
query_img = get_query_img(landmark)

query_features = # TODO: Detect features on the query image
detected_features = []
for img in known_landmark_imgs:
  # TODO
  # detect keypoints and descriptors
  # keypoints, descriptors = ...
  # detected_features.append((keypoints, descriptors))

Now that we have a feature for all image, we can go through them and try to match them using FLANN. Take a look at the introduction text and the references to understand how to do this.

In [ ]:
matched_fnames = []

for image_fname, features in zip(known_landmark_fnames, detected_features):
  keypoints, descriptors = features
  # TODO: Perform matching between the query image and the current image using FLANN and ratio test.
  # Note: You might get an error in the line ̨ `kp1, kp2 in matches` if there are no matches.
  # Instead, use `match in matches` and check if `len(matches) < 2`. If so, skip the image.
  # Note: Feel free to change the matches threshold to get better results (capture more matches).

  is_match = False
  if is_match:
    matched_fnames.append(image_fname)

print('Matched images:', len(matched_fnames))
print('Known matches:', len(known_landmark_fnames))

Now you have implemented the image matchign process for a limited subset of the data. Next, you will implement the same algorithm for all images in the dataset.

## Run the Algorithm on All Images

In [ ]:
# TODO: Implement the specified feature detection method for all images.

query_image = # TODO
query_features = # TODO
img_features = []

for i in range(num_imgs):
  img = get_img(i)
  img_fname = get_img_fname(i)
  # TODO
  # detect keypoints and descriptors as earlier

matched_fnames = []
# TODO: Perform matching between the query image and all images using FLANN



Now that we have the results on the whole dataset, we can evaluate the performance of the algorithm.

## Evaluation

For evaluation, you will calculate values of the confusion matrix.

https://en.wikipedia.org/wiki/Confusion_matrix

True positives: Landmark is detected by your algorithm and is really a landmark; i.e. landmark is in both `matched_fnames` and `known_landmark_fnames`.

False positives: Landmark is matched by your algorithm but in reality the image does not contain the landmark; i.e. landmark is in `matched_fnames` but not in `known_landmark_fnames`.

In [ ]:
# Calculate number of true positives and false positives
#
# True positives: Landmark is in both matched_fnames and known_landmark_fnames
# False positives: Landmark is in matched_fnames but not in known_landmark_fnames

def get_confusion_matrix(matched_fnames, known_landmark_fnames, num_imgs):
  '''
  matched_fnames: Your retrieval results from all images.
  known_landmark_fnames: Known GT landmark names.
  '''
  # TODO: Calculate the values
  tp = 0
  fp = 0
  fn = 0
  tn = num_imgs - (tp + fp + fn)
  return (tp, fp, tn, fn)

# Test if confusion matrix works correctly
matched_fnames_test = ['eiffel_0, eiffel_1, eiffel_2, louvre_0']
known_landmark_fnames_test = ['eiffel_0', 'eiffel_1', 'eiffel_2', 'eiffel_3']
# (tp, fp, tn, fn)
test_cm = (3, 1, num_imgs - (3 + 1 + 1), 1)
assert test_cm == get_confusion_matrix(matched_fnames_test, known_landmark_fnames_test, num_imgs),\
  "Your function is not implemented correctly."

In [ ]:
# Print the confusion matrix values

tp, fp, tn, fn = get_confusion_matrix(matched_fnames, known_landmark_fnames, num_imgs)
print(f'{tp}\t{fp}')
print(f'{fn}\t{tn}')

**Final step:** Try to modify your matches threshold to increase the number of true positives.